# Buscador

Esse notebook implementa um buscador simples. 
A representação pra cada texto é criada a partir da TF-IDF.
A representação da query (consulta, ou termos buscados)
é construída a partir do vocabulário dos textos.
O ranqueamento dos resultados é feito de acordo com
a semelhança cosseno da query pros textos.

Há várias oportunidades de melhoria. 
Algumas delas são discutidas ao longo do notebook.

Os resultados, mesmo deste buscador ingênuo,
são bastante satisfatórios.
O buscador é capaz de retornar leis (neste caso)
relacionadas à localidades ou personalidades.
No entanto, o mesmo mecanismo pode ser utilizado
pra quaisquer outros textos, por exemplo o Diário Oficial.
Alguns exemplos de buscas são: 

"winterianus" - retorna a Lei Municipal sobre citronelas;

"Elydio Azevedo" - retorna Lei Municipal que concede título de cidadão feirense;

"Rua Espassonavel" - retorna Lei Municipal que cita a rua.

In [6]:
import pandas as pd

In [7]:
laws = pd.read_json('leis.json')
laws.drop(['documento'], inplace=True, axis=1)
print(laws.info())
print(laws.nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6033 entries, 0 to 6032
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   titulo     6033 non-null   object
 1   categoria  6033 non-null   object
 2   resumo     6033 non-null   object
 3   texto      6033 non-null   object
dtypes: object(4)
memory usage: 188.7+ KB
None
titulo       6033
categoria       8
resumo       4961
texto        6029
dtype: int64


In [8]:
laws

,titulo,categoria,resumo,texto
0,"DECRETO Nº 8854, de 28 de fevereiro de 2013.",Decretos,DELEGA COMPETÊNCIA À SECRETARIA MUNICIPAL DE P...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
1,"DECRETO Nº 8849, de 25 de fevereiro de 2013.",Decretos,ABRE CRÉDITO SUPLEMENTAR AO ORÇAMENTO DO MUNIC...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
2,"DECRETO Nº 8853, de 27 de fevereiro de 2013.",Decretos,NOMEIA MEMBROS DO CONSELHO MUNICIPAL DE DESENV...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
3,"DECRETO Nº 8967, de 17 de julho de 2013",Decretos,ALTERA O QUADRO DE DETALHAMENTO DE DESPESA DO ...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
4,"DECRETO Nº 8982, de 30 de julho de 2013",Decretos,AUTORIZA O FUNCIONAMENTO DE ESTABELECIMENTOS C...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
...,...,...,...,...
6028,RESOLUÇÃO Nº 125/1980,Resoluções,DISPÕE SOBRE A CONCESSÃO DE TÍTULO DE CIDADÃO ...,Faço saber que a Câmara Municipal aprovou e eu...
6029,RESOLUÇÃO Nº 403/2003,Resoluções,AUTORIZA A MESA DIRETIVA DO PODER LEGISLATIVO ...,"A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado..."
6030,RESOLUÇÃO Nº 492/2014,Resoluções,INSTITUI A SEGUNDA SEMANA DO MÊS DE AGOSTO EM ...,"A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado..."
6031,RESOLUÇÃO Nº 382/2001,Resoluções,CRIA A MEDALHA VEREADOR DIVAL FIGUEIREDO MACHA...,"A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado..."


In [9]:
print(laws.loc[len(laws)-1, 'texto'])

A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado da Bahia, na conformidade do artigo 70, Inciso V, da Lei Municipal nº37, de 05 de Abril de 1990 e, artigos 287, § 2º e, 420, do Regimento Interno, promulga a seguinte Resolução:

Art. 1ºDê-se aos dispositivos abaixo mencionados, da Resolução nº393/2002 - Regimento Interno, as seguintes redações:

"Art. 7º A Mesa Diretora da Câmara compor-se-á do Presidente, Primeiro e Segundo Secretários, com mandato de 02 ( dois ) anos, admitida a recondução para a eleição subsequente.

§ 4º Se, hora regimental, não estiver presente o Presidente, abrirá os trabalhos o Vice-Presidente ou, na falta deste, o Primeiro ou Segundo Secretários, na sequência, ou ainda, caso estes não estejam presentes, o Vereador mais votado nas eleições municipais."

"Art. 33 Compete, privativamente, ao Vice-Presidente:"

"Art. 36 ...

I - ...

e) acompanhar e supervisionar a Ata da Sessão, proceder a sua leitura e assiná-la depois do Presidente e do Vice-Presidente.

II - ...



# Buscas por texto

No notebook _similar_laws_ vimos que TF-IDF encontra Leis bastante similares entre si.
Será que conseguimos também encontrar leis similares a uma query?

Primeiro, devemos construir a representação das leis com TF-IDF.
Após termos as representações,
limpamos o texto da consulta utilizando o mesmo método de limpeza das leis.
Depois, criar uma representação da consulta utilizando o IDF do modelo treinado.
Finalmente, calcular a similaridade desta consulta
para todas as leis na base e retornar as mais próximas.

In [5]:
from scripts.parsers import clean_text
laws['texto_limpo'] = laws['texto'].apply(clean_text)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(laws['texto_limpo'])
X

<6033x30209 sparse matrix of type '<class 'numpy.int64'>'
	with 625873 stored elements in Compressed Sparse Row format>

In [7]:
transformer = TfidfTransformer()
X_tfidf = transformer.fit_transform(X)

X_tfidf

<6033x30209 sparse matrix of type '<class 'numpy.float64'>'
	with 625873 stored elements in Compressed Sparse Row format>

In [21]:
query = ['rua espassonavel']
query[0] = clean_text(query[0])
query = vectorizer.transform(query)
query = transformer.transform(query)

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

best_matches = cosine_similarity(query, X_tfidf)
best_matches_idx = np.argsort(best_matches)
for i in range(1,5):
    idx = best_matches_idx[0, -i]
    print(laws.loc[idx, 'texto'])
    print('\n---Next Result:---\n')

Autor: Cintia Daltro Machado

O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Estado da Bahia, FAÇO saber que a Câmara Municipal, através do Projeto de Lei nº 158/2007, de autoria da Edil Cíntia Daltro Machado, decretou e eu sanciono a seguinte Lei:

Art. 1ºFica considerada de utilidade pública a IGREJA EVANGÉLICA PENTECOSTAL MONTE CARMELO, com sede provisória na Rua Espassonavel, nº 195, Bairro George Américo, neste Município de Feira de Santana.

Art. 2ºEsta Lei entrará em vigor na data de sua publicação, revogadas as disposições em contrário.

PREFEITURA MUNICIPAL DE FEIRA DE SANTANA, em 27 de Fevereiro de 2008.

JOSÉ RONALDO DE CARVALHO
Prefeito

---Next Result:---

O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Estado da Bahia, no uso de suas atribuições. Faço saber que a Câmara Municipal, através do Projeto de Lei nº 163/2017, deste Poder Executivo, decretou e eu sanciono a seguinte Lei:

Art. 1ºOs Logradouros do Conjunto Habitacional Núcleo Conceição, localizados ao lado direito da Rua Ita

Tcharam! Feito um buscador simples!

Existem limitações.
A sequência e composição das palavras é uma delas, por exemplo.
Não adianta buscar pelo nome - sobrenome de uma pessoa.
Ele vai retornar resultados onde
algum destes termos sejam mais frequentes.
Não existe as aspas do Google pra dizer
"busque por este termo todo junto".

Por exemplo, se eu buscar Elydio,
o primeiro resultado é a Lei conferindo
cidadania à Elydio Azevedo Lopes.
Perfeito.
Mas se eu buscar Azevedo Lopes,
o primeiro resultado sequer tem Azevedo,
mas o nome Lopes aparece mais de uma vez.

Uma das formas de contornar essa dificuldade é
usar bigramas ou n-gramas maiores.


## Outras opções
### Indexar
Há outras formas de indexar os documentos
e de recuperar, também simples.
Uma outra forma de indexar, por exemplo,
é fazer um vetor pra cada palavra
contando as palavras vizinhas.
E depois, o vetor do documento seria
a soma dos vetores das palavras.
É uma forma interessante porque
pode gerar visualizações interessantes
entre a similaridade das palavras.
Por exemplo, no corpus das Leis Municipais,
a quais palavras EDUCAÇÃO mais se assemelha?
Ou SAÚDE? Etc.

Outra forma é contar n-gramas - por exemplo,
bi-gramas: duas palavras juntas formando um token.
Dessa forma, você possui uma matriz maior
e de certa forma uma relação entre a sequencialidade das palavras,
que pode ser útil pra nomes de pessoas e bairros,
como citado acima.

### Recuperar
Outra forma de recuperar é por
_local sensitive hashing_.
Divide em vários planos múltiplas vezes
e retorna os resultados que estão na mesma região da query.
No entanto,
o corpus não é grande o suficiente pra precisar essa estratégia,
que é mais pra grandes corpora.
O método acima
(calcular a simlaridade cosseno e retornar os maiores valores)
é rápido o suficiente pra parecer instantâneo.
Talvez com uma demanda mais alta pelo servidor
venha a necessidade de aumentar a velocidade da busca,
porém por enquanto não é o caso.

Há ainda um [novo método]
(https://ai.googleblog.com/2020/07/announcing-scann-efficient-vector.html)
e uma lib pra isso,
lançada pelo Google recentemente,
no dia 28 de Julho de 2020.

### Avaliação
Com múltiplas formas de indexar e recuperar vem o dilema:
como avaliar se uma é melhor que a outra?
Repetir o processo acima pra todas as opções?
Isto é, mostrar N melhores resultados e comparar manualmente?
Ou colocar labels em algumas leis?
Ex: essa lei trata disso, com tais entidades.
Checar formas de avaliação.
Se tivesse em produção,
poderiamos avaliar por _click through rate_ (CTR) por ex,
mas não é o caso